In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import matplotlib.pyplot as plt
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
from behav_proc import addBehavCols
import numpy as np

ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)


#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')

data_dir_input = os.path.expandvars('$HOME/ownCloud/Current/merr_data/Bonaiuto')

#/home/demitau/Downloads/all_data_modified.csv

import matplotlib as mpl
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

tgtc = ['b','g','r','brown']

import seaborn as sns
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

group 0: implicit, 1:explicit
block:
1 - short training block (could be discarded, lots of mistakes, bad trials)
2 - baseline block (zero coherence, no perturbation)
3-8 - trial blocks (with coherence and direction changes and perturbation dependent on the group assignment)
9 - washout block (zero coherence, no perturbation)

trial_type:
-1 - training trials
0 - baseline trials
1 - perturbation trials
2 - washout trials

reach_target: angle of the trajectory at the target distance, relative to the target
(DT: as far as I understad it means AFTER pert is applied)

reach_real_angle: absolute angle of the trajectory at the target distance

perturb_cat: direction of the perturbation

reach_sub_perturb: reach_target - perturb_cat


In [ ]:
df_all0 = pd.read_csv( pjoin(data_dir_input,'all_data_modified.csv'))
df_all = df_all0.copy().rename(columns={'subject_id':'subject', 
                                        'trial_target':'target_inds',
                    'perturb_cat':'perturbation', 
            'true_target_angle':'target_locs'})
df_all['error'] = \
    df_all0['reach_real_angle'] - df_all0['perturb_cat'] - df_all0['true_target_angle']
df_all['feedback']     = df_all['reach_real_angle'] + df_all['perturbation']
df_all['org_feedback'] = df_all['reach_real_angle'] 


subjects = df_all['subject'].unique().tolist(); subjects 


blockid2type = {1:'short_train', 2:'baseline', 9:'washout'}
blockid2name = {1:'short_train', 2:'baseline', 9:'washout'}
for i in range(3,9):
    blockid2type[i] = 'perturbation'
    blockid2name[i] = 'perturbation' + str(i-2)
# trial type would be more appropriate column name, more consistent with Maciej names
df_all['block_type'] = df_all['block'].apply(lambda x: blockid2type[x]) 
df_all['block_name'] = df_all['block'].apply(lambda x: blockid2name[x]) 

df_all['trials'] = -1
bl = 56
for i,row in df_all.iterrows():
    bi = row['block']
    offset = 0
    if bi > 1:
        offset += 12
        offset += bl * (bi-2)
    df_all.loc[i,'trials'] = offset + row['trial_in_block']
    
tgt_inds_all = list( df_all['target_inds'].unique() )  + [None]; print(tgt_inds_all)

! fixate 0.6 x 0.6 deg -- bullseye and crosshairs,
square cursor 0.5 x 0.5 deg
variable dealy (1-2_s)
RDK presentation together with five tgts (from -120 to 0)
the one of three (-90, -30)

3.25 deg -- larger tgt


In [ ]:
#df_all.query(f'block in [1,2] and subject == "{subjects[0]}" '
#             'and trials <= 100 and trials > 10')[['trials',
#                                            'target_locs','reach_target']]

In [ ]:
#%debug
#dfs0 = df_all.query('subject == @subjects[1]')
df_all = addBehavCols(df_all, inplace=1, skip_existing=1, dset = 'Bonaiuto')

target_size = 3.25
def f(row):
    #print(row.keys())
    target_loc = row['target_locs']
    feedback_ang  = row['feedback']
    ae = abs(target_loc - feedback_ang)
    nh = ae > target_size
    #print(nh, ae,target_size)
    return nh

df_all['non_hit_not_adj'] = df_all.apply(f,1)

In [ ]:
from behav_proc import computeErrSensVersions
#envs_cur = ['stable', 'random', 'all']
envs_cur = ['all']
tgt_inds_cur = tgt_inds_all
#tgt_inds_cur = [None]  # DEBUG

block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ None ]
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'

df_all['environment'] = 1111
df_all['feedbackX'] = 1
df_all['feedbackY'] = 1
#%debug
df_all_multi_tsz, ndf2vn = computeErrSensVersions(df_all, envs_cur,block_names_cur,
                pertvals_cur,gseqcs_cur,tgt_inds_cur,
               dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
                                subj_list = subjects, error_type=error_type,
                                trial_shift_sizes = np.arange(1,5),
                                         addvars=[], target_info_type = 'locs')

assert not df_all_multi_tsz.duplicated().any()  
#display( dftmp2.duplicated()  )
assert not df_all_multi_tsz.duplicated(['subject','trials','trial_shift_size',
                             'trial_group_col_calc']).any()

In [ ]:
#prop non-hit
dftmp = df_all.query('subject == @subjects[1]')
sum( ~dftmp['non_hit_not_adj'])  / len( dftmp )

In [ ]:
df_all0.query('block in [1,2]').groupby(['block']).size()

In [ ]:
df_all0.query('block in [1,2]').groupby(['block','trial_in_block']).size()

## plots for bonaituo bias discuss

In [ ]:
sns.__version__

In [ ]:
df_all['error'] = (df_all['feedback'] - df_all['target_locs'] )

In [ ]:
df_all['error'] = \
    df_all0['reach_real_angle'] - df_all0['perturb_cat'] - df_all0['true_target_angle']

In [ ]:
df_all['tmpe'] = (df_all['feedback'] - df_all['target_locs'] )

df_all['tmpe'] = df_all['reach_sub_perturb']

qs = 'block in [1,2, 3,4,5,6,7,8,9] and group == 0'
ax = sns.lineplot(data=df_all.query(qs), 
             y='tmpe', x='trials', errorbar='se')
ax.axhline(y=0, ls=':', c='r', lw=3)
ax.set_title(f'block in {qs}')

In [ ]:
# per subj first two blocks
plt.figure(figsize=(20,20))
df_all0['error'] = \
    df_all0['reach_real_angle'] + df_all0['perturb_cat'] - df_all0['true_target_angle']
df_ = df_all0.query('block in [1,2]')
fg = sns.catplot(data=df_, kind='violin', y='error',
                   x='block', col_wrap = 5, errobar='se',col='subject_id')

axs = fg.axes
for ax in axs.flatten():
    ax.axhline(y=0, ls=':', c='r', lw=3)
    ax.set_ylim(-100,90)
#plt.

In [ ]:
#plt.figure(figsize=(40,7))
ax = sns.violinplot(data=df_all.query('block in [1,2]'), y='error',x='block')
ax.set_ylim(-100,90)
ax.axhline(y=0, ls=':', c='r', lw=3)
ax.set_title('block in [1,2]')

In [ ]:
qs = 'block in [1,2, 3,4,5,6,7,8,9] and group == 0'
ax = sns.lineplot(data=df_all.query(qs), 
             y='error', x='trials', errorbar='se')
ax.axhline(y=0, ls=':', c='r', lw=3)
ax.set_title(f'block in {qs}')

In [ ]:
import gc; gc.collect()

In [ ]:
df_all['target_locs'].apply(lambda x: f'{x:.2f}')

In [ ]:
dftmp = df_all.query('subject == @subjects[1]')
sum( ~dftmp['non_hit_not_adj'])  / len( dftmp )

In [ ]:
df_all_multi_tsz.groupby(['subject','trial_shift_size',
                             'trial_group_col_calc'] ).size()

In [ ]:
dftmp = df_all_multi_tsz.query('subject == @subjects[1]')
sum( np.isinf(dftmp['err_sens']) ) / len( dftmp )

In [ ]:
dftmp['err_sens']

In [ ]:
df_all_multi_tsz['dist_rad_from_prevtgt2'].unique()

In [ ]:
np.isinf(df_all_multi_tsz['err_sens'] ).all()

In [ ]:
df_all_multi_tsz['err_sens'].to_numpy()

In [ ]:
df_all.columns

In [ ]:
#df_all[['target_locs','distsgn_rad_from_prevtgt']]

In [ ]:
# check trial number variation
nums = df_all.groupby(['subject','block']).size().reset_index()
nums.groupby(['block']).mean(numeric_only=1), nums.groupby(['block']).std(numeric_only=1)

In [ ]:
dfs0 = df_all.query('subject == @subjects[1]')
dfs0[['subject','group','block','block_type','trials','trial_in_block','perturbation', 'error']]

In [ ]:
#dfs0.plot('trials')

In [ ]:
df_all_multi_tsz.columns

In [ ]:
#df_all_multi_tsz['trial_group_col_calc']

In [ ]:
#df_all_multi_tsz.groupby

In [ ]:
#sns.lineplot?

In [ ]:
#'reach_target':'feedback_ang'}
# + df_all['target_locs']
#df_all['feedback']     = df_all['reach_real_angle'] + df_all['perturbation']
#df_all['org_feedback'] = df_all['reach_real_angle'] 
#df_all['tst']     = df_all['reach_real_angle'] - 60

#df_ = df_all.query('subject == @subjects[1] and block > 1')
df_ = df_all.query('group == 0 and block > 1')
dfone = df_.query('subject == @subjects[1]')

ebarg = ''
ax = sns.lineplot(x="trials", y='feedback',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

plt.figure()
ax = sns.lineplot(x="trials", y='org_feedback',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

plt.figure()
ax = sns.lineplot(x="trials", y='error',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

In [ ]:
# TODO: per particpant

In [ ]:
len(subjects)

In [ ]:
import seaborn as sns
#'reach_target':'feedback_ang'}
# + df_all['target_locs']
#df_all['feedback']     = df_all['reach_real_angle'] + df_all['perturbation']
#df_all['org_feedback'] = df_all['reach_real_angle'] 
#df_all['tst']     = df_all['reach_real_angle'] - 60

#df_ = df_all.query('subject == @subjects[1] and block > 1')
df_ = df_all.query('group == 1 and block > 1')
dfone = df_.query('subject == @subjects[0]')

ebarg = ''
ax = sns.lineplot(x="trials", y='feedback',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

plt.figure()
ax = sns.lineplot(x="trials", y='org_feedback',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

plt.figure()
ax = sns.lineplot(x="trials", y='error',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

In [ ]:
sns.lineplot?

In [ ]:
df_.groupby(['trials']).size()

In [ ]:
dfc_['err_sens']

In [ ]:
coln = 'err_sens'
#debug
from behav_proc import truncateDf
dfc_multi_tsz = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard')
dfc_multi_tsz = dfc_multi_tsz.query('dist_rad_from_prevtgt2 != "nan"')

In [ ]:
def fg(cohv):
    if cohv <= 0.33:
        return 0
    elif cohv <= 0.66:
        return 1
    else:
        return 2

dfc_multi_tsz['cohg'] = dfc_multi_tsz['trial_coherence'].apply(fg,1)

In [ ]:
dfc_multi_tsz.query('dist_rad_from_prevtgt2 == "nan" and subject == @subjects[0]')\
[['trials','block','trial_in_block','target_inds','trial_group_col_calc','trial_shift_size']]

In [ ]:
df_all_multi_tsz.groupby(['trials','trial_shift_size', 
                 'dist_rad_from_prevtgt2',
                 'trial_group_col_calc','group'] ).size()

In [ ]:
from config2 import path_fig

In [ ]:
plt.hist( df_['trial_coherence'].unique() )

In [ ]:
rp.summarize(df_.groupby(['group']) [['err_sens']])

In [ ]:
rp.summarize(df_.groupby(['group', 'cohg']) [['err_sens']])

In [ ]:
import researchpy as rp
# 0 > 1 statistically
rp.difference_test("err_sens ~ C(group)",
                   data = df_,
                   equal_variances = False,
                   independent_samples = True).conduct(effect_size = "all")

In [ ]:
import researchpy as rp
# 0 > 1 statistically
rp.difference_test("err_sens ~ C(cohg)",
                   data = df_.query('cohg != 1'),
                   equal_variances = False,
                   independent_samples = True).conduct(effect_size = "all")

In [ ]:
import researchpy as rp
# 0 > 1 statistically
for g in [0,1]:
    print(g)
    r = rp.difference_test("err_sens ~ C(cohg)",
                       data = df_.query(f'cohg != 1 and group == {g}'),
                       equal_variances = False,
                       independent_samples = True).conduct(effect_size = "all")
    display(r)

In [ ]:
g2vs = {}
for g in [0,1]:
    g2vs[g] = df_.query('group == @g')['err_sens']._values

In [ ]:
tts

In [ ]:
df_ = dfc_multi_tsz.query('trial_shift_size == 1' 
                            ' and trial_group_col_calc == "trials"')

In [ ]:
ax = sns.violinplot(data=df_,
                 y='err_sens',                                   
                 x = 'group')
ax.set_ylim(-7.5,7.5)
ax.axhline(0,c='r',ls=':')

In [ ]:
fg = sns.catplot(kind='violin', data=dfc_multi_tsz.query('trial_shift_size == 1' 
                            ' and trial_group_col_calc == "trials"'),
                 y='err_sens',                  
                 #x = 'dist_rad_from_prevtgt2',
                 x='cohg',
                 hue = 'group', split=True)
for ax in fg.axes.flatten():
    ax.set_ylim(-4,5)
    ax.axhline(0,c='r',ls=':')

In [ ]:
fg = sns.catplot(kind='violin', data=dfc_multi_tsz.query('trial_shift_size == 1' 
                            ' and trial_group_col_calc == "trials"'),
                 y='err_sens',                  
                 x = 'dist_rad_from_prevtgt2',
                 hue = 'group', split=True)
for ax in fg.axes.flatten():
    ax.set_ylim(-4,6)
    ax.axhline(0,c='r',ls=':')

In [ ]:
#dist_rad_from_prevtgt2
import seaborn as sns
fg = sns.catplot(kind='violin', data=dfc_multi_tsz, y='err_sens', 
                 col = 'trial_shift_size', 
                 row = 'dist_rad_from_prevtgt2',
                 hue = 'group',
                     x='trial_group_col_calc', split=True)
for ax in fg.axes.flatten():
    ax.set_ylim((-5,7))
    ax.axhline(y=0, c='r', ls=':')
plt.savefig(pjoin(path_fig,'behav','bonaiuto_summary.pdf'))

In [ ]:
from behav_proc import truncateDf
#df_ = df_all_multi_tsz.query('trial_shift_size == 1 and group == 0 and block > 1')
#dfc_  = truncateDf(df_, 'err_sens', q=0.05, clean_infnan=True)

for grp, subj in [(0, subjects[1]), (1, subjects[0])]:
    qs = 'trial_shift_size == 1 and group == @grp and block > 1'
    dfc_ = dfc_multi_tsz.query(qs)
    dfone = df_all_multi_tsz.query(qs + ' and subject == @subj')

    display( dfc_.groupby(['trials','trial_group_col_calc']).size() )

    plt.figure(figsize=(12,5))
    ax = sns.lineplot(x="trials", y='err_sens',data=dfc_,  estimator='mean', 
                      errorbar='se',
                     hue = 'trial_group_col_calc')#, hue='block')
    ax.set_ylim(-1.5,2.5)
    ax.axhline(0,ls=':',c='red')
    ax.plot(dfone['trials'].to_numpy(), 
            dfone['perturbation'].to_numpy() / 30, ls=':',c='y')
    ax.legend(loc='lower right')
    ax.set_title(f'group = {grp}')

# plt.figure()
# ax = sns.lineplot(x="trials", y='tst',data=df_,  estimator='mean', errorbar='se')#, hue='block')
# ax.axhline(0,ls=':',c='red')
# ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')

# ys = dfone['org_feedback'].to_numpy()
# ax.plot(dfone['trials'].to_numpy(), ys, c='orange', alpha = 0.4)

#ys = dfone['feedback'].to_numpy() - dfone['target_locs'] - dfone['perturbation']
#ax.plot(dfone['trials'].to_numpy(), ys, c='g', alpha=0.3)
#,  estimator='mean', errorbar='se',

# error = reach_sub_perturb = reach_target - perturb_cat 

In [ ]:
# violin comp

In [ ]:
drs = dfc_multi_tsz['dist_rad_from_prevtgt'].unique();
drs_f = np.array( list(map( float, drs) ) )
drs = list(sorted(drs,key = lambda x: float(x)))

#dfc_s = dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']
# TODO: should I restrict to one env?
dfc_s = dfc_multi_tsz.query('trial_group_col_calc == "trialwe"')
grp = dfc_s.groupby(['trial_shift_size', 'dist_rad_from_prevtgt2'])
dts = dfc_multi_tsz['trial_shift_size'].unique()
nt = len(dts)

drpairs = [(drs[i],drs[i+1] ) for i in range(2)]
#drpairs += [(drs[0],drs[i] ) for i in range(2,4)]
drpairs += [(drs[0],drs[2] )]
drpairs

dtpairs = [(dts[i],dts[i+1] ) for i in range(3)]
dtpairs += [(dts[0],dts[i] ) for i in range(2,4)]
dtpairs += [(dts[1],dts[3] )]
dtpairs, drpairs

In [ ]:
def getPvals(dftmp,fitcol,pairs):
    from scipy.stats import ttest_ind
    pvalues = []
    for drp in pairs:
        if isinstance(drp[0],str):
            vs1 = dftmp.query(f'{fitcol} == "{drp[0]}"')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == "{drp[1]}"')['err_sens']
        else:
            vs1 = dftmp.query(f'{fitcol} == {drp[0]}')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == {drp[1]}')['err_sens']
        #ttr = ttest_ind(vs1,vs2)
        assert not vs1.isna().any(), 'vs1 contains NaN'
        assert not vs2.isna().any(), 'vs2 contains NaN'
        assert len(vs1)
        assert len(vs2)
        ttr = ttest_ind(vs1,vs2, alternative='greater')
        pvalues += [ttr.pvalue]
        
        print('pair = ',drp, 'p = ',ttr.pvalue)
        #print(vs1,vs2)
    # pvalues = [
    #     sci_stats.mannwhitneyu(robots, flight, alternative="two-sided").pvalue,
    #     sci_stats.mannwhitneyu(flight, sound, alternative="two-sided").pvalue,
    #     sci_stats.mannwhitneyu(robots, sound, alternative="two-sided").pvalue
    # ]

    # pvalues
    # [0.00013485140468088997, 0.2557331102364572, 0.00022985464929005115]

    # Transform each p-value to "p=" in scientific notation
    formatted_pvalues = [f'p={pvalue:.2e}' for pvalue in pvalues]
    return pvalues, formatted_pvalues
pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)

In [ ]:
pairs

In [ ]:
fitcol

In [ ]:
dftmp

In [ ]:
# make polynomial fits
def plotPolys(ax, dftmp, fitcol, degs=range(2,6), mean=1):
    if mean:
        me = dftmp.groupby(fitcol).median().reset_index()
        dftmp = me
    dftmp[fitcol] = pd.to_numeric(dftmp[fitcol] )
    esv, dv = dftmp[['err_sens',fitcol]]._values.T
    print(np.min(dv),dv,dv-np.min(dv),esv)
    #pr = np.polyfit(esv,dv,2)
    from numpy.linalg import LinAlgError
    dvu = np.unique(dv)
    dvu = np.array( list(sorted(dvu)) )
    print(f'median of grouped over {fitcol}', dvu)
    for deg in degs:
        try:
            pr = np.polyfit(dv-np.min(dv),esv-np.min(esv),deg)        
        except (SystemError,LinAlgError):
            print(f'Failed deg={deg}')
            print(dv,esv, np.std(dv))
            continue
            
        poly = np.poly1d(pr)
        #if len(degs) > 1:
        if mean:
            lbl = f'polynomial fit of means deg={deg}'
        else:
            lbl = f'polynomial fit deg={deg}'
        #else:
        #    lbl = None
        esv2 = poly(dvu-np.min(dvu)) + np.min(esv)
        print(dvu-np.min(dvu), esv)
        ax.plot(range(len(dvu)) , poly(dvu-np.min(dvu)) + np.min(esv), 
                label=lbl, c='grey', lw=0.85 )
    return pr
    #ax.legend(loc='lower right')

#plotPolys(ax,dftmp,fitcol,mean=0)
ax = plt.gca()
plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)

In [ ]:
qs

In [ ]:
(dfc_multi_tsz['dist_rad_from_prevtgt2'] != '0.00').all()

In [ ]:
#dftmp0['err_sens']

In [ ]:
order

In [ ]:
fitcol

In [ ]:
dfc_multi_tsz[fitcol].unique()

In [ ]:
df_all['target_locs'].unique()

In [ ]:
dfc_multi_tsz.query('group == 1')[fitcol].unique()

In [ ]:
dfc_notna_multi_tsz = dfc_multi_tsz[~dfc_multi_tsz['err_sens'].isna()]

In [ ]:
import seaborn as sns
#%debug

#hue = 'env_nice'
env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-10,10]
meanfit=1
bnqs = [  ('grid_line_violin_spatial','Spatial distance [deg]',drs,drpairs,
           'dist_rad_from_prevtgt2', 
           f'trial_group_col_calc == "trials" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'),
        ( 'grid_line_violin_temporal','Temporal distance [number of trials]',dts,dtpairs,
         'trial_shift_size',
         'trial_group_col_calc == "trials" and dist_rad_from_prevtgt2 == "0.00"')]

#for env_type in ['stable','random']:
    #color = env2color[env_type]
color = 'lightblue'
for bn,xlab,order,pairs,fitcol,qs in bnqs:
    #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
    #qs = qs + ' and trial_shift_size == 1'
    qs += ' and group == 0'

    dftmp0 = dfc_notna_multi_tsz.query(qs)
    
    assert len(dftmp0) > 0
    #fitcol = 'dist_rad_from_prevtgt2'
    fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                     x=fitcol, order = order,
                     split=True, color=color, aspect=aspect)
    dftmp = dfc_notna_multi_tsz.query(qs).copy()
    pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)


    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',c='r')
        ax.set_ylabel('Error sensitivity', fontdict={'fontsize':12})
        ax.set_xlabel(xlab, fontdict={'fontsize':12} )
#         if bn == 'grid_line_violin_spatial':
#             ax.set_xticklabels(['same target', 45, 90, 135])
        ax.set_ylim(ylim)
        ax.set_title(f'{bn}', y = 0.967,
                    fontdict={'fontsize':15, 'fontweight':'bold'})


    from statannotations.Annotator import Annotator    
    annotator = Annotator(ax,pairs, data=dftmp0, x=fitcol, y='err_sens',
                         plot='violinplot', order=order)
    #annotator.set_custom_annotations(formatted_pvalues)
    annotator.set_pvalues(pvalues)
    annotator.annotate()

    ax.set_ylim(-5,17)
    #break # test

    plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
    
    #bn = 'grid_line_violin_spatial'

    #plt.savefig(pjoin(path_fig,'behav',f'{env_type}_{bn}.svg'))
    #plt.savefig(pjoin(path_fig,'behav',f'{env_type}_{bn}.pdf'))



In [ ]:
#df_ = dfs0#.query('block > 1')
df_ = df_all.query('group == 1 and block > 1')
dfone = df_.query('subject == @subjects[0]')
import seaborn as sns
ax = sns.lineplot(x="trials", y='error',data=df_,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')
#,  estimator='mean', errorbar='se',

In [ ]:
from behav_proc import *
addBehavCols(df_all)